In [1]:
import pygame
import random

from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import os


from stable_baselines3 import A2C

pygame 2.4.0 (SDL 2.26.4, Python 3.9.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [16]:
pygame.init()
screen = pygame.display.set_mode((500,500))

clock = pygame.time.Clock()


In [17]:

class PaddleAgent:
    def __init__(self,pos):
        self.rect = pygame.Rect(pos[0],pos[1],15,100)
        self.vel = pygame.Vector2()
        self.vel.x,self.vel.y = 0,0

    def coord_y(self):
        return self.rect.y
    def move(self,action):
        if action == 0:
            self.vel.y  = -5
        elif action == 1:
            self.vel.y =  5
        else:
            self.vel.y  = 0 
        if self.rect.y < 0:
            self.rect.y = 0
        if self.rect.y + self.rect.height > 500:
            self.rect.y = 500-self.rect.height
        self.rect.y += self.vel.y 
    def  draw(self):
        pygame.draw.rect(screen,(255,255,255),self.rect)

   
            
            
class Ball:
    def __init__(self,pos):
        self.col = (255,255,255)
        self.rect = pygame.Rect(pos[0],pos[1],20,20)
        self.vel = pygame.Vector2()
        self.vel.x, self.vel.y = -5,0
    def coord_x(self):
        return self.rect.x
    def coord_y(self):
        return self.rect.y
    def move(self, paddle, opt_paddle):
        c_p1, c_p2  = False,False
        if self.rect.y < 0:
            self.vel.y *= -1
        if self.rect.y + self.rect.height > 500:
            self.vel.y *= -1

        if self.rect.x < 0:
            self.vel.x *= -1
        if self.rect.x + self.rect.width > 500:
            self.vel.x *= -1
        self.rect.y += self.vel.y 
        self.rect.x += self.vel.x


        
        if self.rect.colliderect(paddle.rect):
            self.vel.x *= -1
            distance  = (40 + paddle.rect.y)  - self.rect.y
            if distance > 0:
                
                self.vel.y = (distance/20+2)*-1
                           
            elif distance < 0:
                self.vel.y = (abs(distance)/20+2)
                
            else:
                self.vel.y = self.vel.y * -1
        if self.rect.colliderect(opt_paddle.rect):
          self.vel.x *= -1
          distance  = (40 + opt_paddle.coord_y())  - self.rect.y
          # if distance > 0:
              
          #     self.vel.y = (distance/20+2)*-1
                          
          # elif distance < 0:
          #     self.vel.y = (abs(distance)/20+2)
              
          # else:
          #     self.vel.y = self.vel.y * -1
          self.vel.y = (distance/20+2)*random.choice([-1,1])

              
    def draw(self):
        pygame.draw.rect(screen,self.col,self.rect)


class pong_env(Env):
    def __init__(self, agent):
        # create the game
        self.ball = Ball((250,250))
        self.agent = agent
        self.opt_paddle = PaddleAgent([15,200])
        self.state = [self.ball.coord_x(), self.ball.coord_y(), self.agent.coord_y()]
        # Actions, up, down, leave
        self.action_space = Discrete(3)
        # The x and y coordinate of the ball
        self.observation_space = Box(low = np.array([0,0,0]), high = np.array([500,500,500]))
        
    def step(self, action):
        if self.ball.coord_y()  > self.opt_paddle.coord_y()+(self.opt_paddle.rect.height//2) + random.choice([-1,1]) * 10:
           self.opt_paddle.move(1)
        if self.ball.coord_y()  < self.opt_paddle.coord_y()+(self.opt_paddle.rect.height//2) + random.choice([-1,1]) * 10:
           self.opt_paddle.move(0)
        
        
        self.agent.move(action)
        self.ball.move(self.agent,self.opt_paddle)
        self.state = [self.ball.coord_x(), self.ball.coord_y(), self.agent.coord_y()]
        done = False
        
        reward = 0
        if (abs(self.ball.coord_y()-self.agent.coord_y()) > 10):
          reward = -5
        else:
          reward = 1
        if self.ball.rect.colliderect(self.agent.rect):
          reward += 5

        if self.ball.coord_x() > 460:
          done = True

        if self.ball.coord_x() < 15:
          done = True

        info = {}
        return np.array(self.state), reward, done, False,info
    def render(self):
        screen.fill((0,0,0))
        self.agent.draw()
        self.opt_paddle.draw()
        self.ball.draw()
        pygame.display.update()
        clock.tick(60)


    def reset(self):
        self.opt_paddle.x, self.opt_paddle.y = 15,200
        self.ball.rect.x, self.ball.rect.y = 250,250
        self.ball.vel.x, self.ball.vel.y =  -5, 0
        self.agent.rect.x, self.agent.rect.y = 460,200
        self.state = [self.ball.coord_x(), self.ball.coord_y(), self.agent.coord_y()]
        return np.array(self.state), {}


In [18]:
agent = PaddleAgent([460,200])
env = pong_env(agent)

c:\Users\timli\anaconda3\envs\csa\lib\site-packages\gym\spaces\box.py:127: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(f"Box bound precision lowered by casting to {self.dtype}")


In [20]:
model = A2C("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=50000)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 133      |
|    ep_rew_mean        | -566     |
| time/                 |          |
|    fps                | 814      |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -1.01    |
|    explained_variance | 0.000324 |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | -18.4    |
|    value_loss         | 252      |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 217      |
|    ep_rew_mean        | -955     |
| time/                 |          |
|    fps                | 789      |
|    iterations         | 200      |
|    time_elapsed 

In [8]:
model.save("/content/sb_models")
loaded_model = A2C.load("/content/sb_models")

In [21]:
run = True
state,_  = env.reset()
while run:
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False
    action,_  = model.predict(state)
    next_state,reward, done,_, info = env.step(action)
    env.render()
    state = next_state
    if done:
        state,_ = env.reset()
